Homework 1 - Part 2
====

Use the scaffold of the code below to implement 

* a way to read in the CPU data

* a strategy based on the data to choose an attribute

Make sure to pay attention to whether the attribute is better when it has a high (e.g., speed) or a low (e.g., price) value. For a given attribute you can check the dictionary defined below to see whether a larger or a smaller value wins.

In [59]:
larger_is_better = {'clock_speed': True, 
                    'bus_speed': True, 
                    'year': False, 
                    'n_transistors': True, 
                    'data_width': True,  
                    'process': False, 
                    'die_size': False, 
                    'tdp': False}

### Ex. 2.1: Read in the data (5 pts)

In Part 1 of the homework you removed an outlier from the dataset. Your code from Part 1 should write the corrected dataset to a new CSV file.

Below, read in the corrected data file and store it as a Pandas `DataFrame` variable `cards`.

In [60]:
import numpy as np
import pandas as pd
import random

# Your code here

### Ex. 2.2: Implement the strategy (45 pts)


Implement a strategy to beat the opponents. We first provide two baseline strategies, `RandomStrategy` and `MeanStrategy`.

Your strategy has to implement the `pick_attribute` method, which is where your strategy decides what attribute to play. 

In addition, you may choose to implement the `update` and `new_game` method, which keep you informed about the how the game is progressing. These methods are needed for advanced strategies only, and you will have no trouble beating the baselines without them. If, on the other hand, you want your strategy to beat all of your fellow student's strategies, this is where you turn.

In [61]:
class Strategy():

    def __init__(self, cards):
        self.cards = cards
        
    def update(self, outcome, cache):
        pass
    
    def new_game(self):
        pass
        
class RandomStrategy(Strategy):

    def pick_attribute(self, card):
        """
        Pick an attribute at random.
        """
        return random.choice(list(card.keys()))
        
class MeanStrategy(Strategy):
    
    def pick_attribute(self, card):
        """
        Identify attributes in which the current card has a value 
        below the mean (if lower is better) or 
        above the mean (if higher is better).

        Return a random attribute from the set of attributes satisfying the above.
        """
        attributes = list(card.keys())

        # Add attributes with above/below mean to the `eligible` list
        eligible = []
        mean_values = self.cards.mean()
        for attribute in attributes:
            if larger_is_better[attribute]:
                if card[attribute] > mean_values[attribute]:
                    eligible.append(attribute)
            else:
                if card[attribute] < mean_values[attribute]:
                    eligible.append(attribute)

        # Choose randomly from the set of eligible attributes. 
        # If no attribute was eligible, return a totally random attribute.
        if len(eligible):
            return random.choice(eligible)
        else:
            return random.choice(attributes)

Your own strategy should go below

In [62]:
class CunningStrategy(Strategy):
    def pick_attribute(self, card):
        """
        Describe your winning strategy here.
        
        You probably want to change the implementation of the function as well.
        """
        return 'clock_speed'

The code below provides the `play_game` function. You should not change anything in this block. Read through it to make sure that you understand how the game works. 

In [63]:
cards = pd.read_csv("cpu_wars.csv")


In [70]:
def void_update_func(event, cards, turn_i):
    """don't do anything"""
    pass


def play_game(my_strategy, opponent_strategy, cards, N=100):
    """
    Play N games and report statistics.
    """
    wins = 0
    no_of_draws = 0
    no_of_turns = 0
    
    # Take out `name` column 
    cards = cards[cards.columns.difference(['name'])]
    
    # Build a list of dictionaries, each dictionary 
    # representing a single card.
    card_list = cards.to_dict('records')
    
    for round_index in range(N):
        # Notify the strategies that a new game has begun
        my_strategy.new_game()
        opponent_strategy.new_game()
        
        # Shuffle cards
        random.shuffle(card_list)
    
        # Divide into two stacks. 
        # `my_stack` is formed by taking every second card, starting from the first card.
        # `opponent_stack` is formed by taking every second card, starting from the second card.
        my_stack = card_list[::2]
        opponent_stack = card_list[1::2]
    
        # Pick starter
        my_turn = random.random() >= 0.5
    
        cache = []
        
        # Play until one player runs out of cards
        while my_stack and opponent_stack:
            no_of_turns += 1
            
            # Draw cards from the stacks
            my_card = my_stack.pop(0)
            opponent_card = opponent_stack.pop(0)
        
            # Decide attribute
            if my_turn:
                chosen_attribute = my_strategy.pick_attribute(my_card)
            else:
                chosen_attribute = opponent_strategy.pick_attribute(opponent_card)
    
            my_value = my_card[chosen_attribute]
            opponent_value = opponent_card[chosen_attribute]
    
            # Add cards to cache
            cache += [my_card, opponent_card]
    
            # In case both cards have the same value there is a draw
            if my_value == opponent_value:
                my_strategy.update('draw', cache)
                opponent_strategy.update('draw', cache)
                no_of_draws += 1
            else:
                # Find out who is the winner
                if larger_is_better[chosen_attribute]:
                    i_win_turn = my_value > opponent_value
                else: 
                    i_win_turn = my_value < opponent_value
                    
                if i_win_turn:
                    my_strategy.update('win', cache)
                    opponent_strategy.update('lose', cache)

                    random.shuffle(cache)
                    my_stack += cache
                else:
                    my_strategy.update('lose', cache)
                    opponent_strategy.update('win', cache)
                
                    random.shuffle(cache)
                    opponent_stack += cache

                my_turn = i_win_turn
                cache.clear()
                    
        # If I have cards left, I win the game
        if my_stack:
            wins += 1
    
    return {"Success rate": (wins / N), 
            "Avg #turns": (no_of_turns / N),
            "Draw rate": (no_of_draws / N)}
    

Finally, play the game using strategies of your choice. 

The example code below plays 1000 games between two random strategies.

In [85]:
play_game(RandomStrategy(cards), RandomStrategy(cards), cards, N=1000)

{'Avg #turns': 198.334, 'Draw rate': 10.711, 'Success rate': 0.522}

Ex 2.3: Confusion matrix (10 pts)
----

Make a confusion matrix in `pandas` that shows how your strategy, the random, and the mean opponent perform against each other.

In [86]:
# Your code here